In [1]:
import pandas as pd
import psycopg2

In [ ]:
def create_db():
    
    #Connect to database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    print("Database connection successful!")
    cur = conn.cursor()
    conn.set_session(autocommit=True)
    
    #Set Sparkify database with utf-8
    #cur.execute("DROP DATABASE IF EXISTS accounts")
    #cur.execute("CREATE DATABASE accounts")
    
    
    #close default db connection
    conn.close()
    
    #Now connect to sparkify db
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=root")
    print("New database connection successful!")
    cur = conn.cursor()
    #conn.set_session(autocommit=True)
    
    return cur, conn
    
    
    

In [10]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()        

In [11]:
def create_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [18]:
dfAC = pd.read_csv("data/wealthdata.csv")

In [ ]:
dfAC.head()

In [192]:
dfAC_clean = dfAC[["Series Code", "Country Code", "1995 [YR1995]", "1997 [YR1997]", "1999 [YR1999]"]]

In [193]:
dfAC_clean.head()

,Series Code,Country Code,1995 [YR1995],1997 [YR1997],1999 [YR1999]
0,NW.HCA.TO,ALB,44861001529.9015,37131131818.8883,42196257451.8409
1,NW.HCA.PC,ALB,14072.7858380309,11794.0971021609,13573.2617291556
2,NW.HCA.FEMP.PC,ALB,3468.85762831714,2868.99731747057,3259.89287632051
3,NW.HCA.MEMP.PC,ALB,6507.93123327619,5466.6915395155,6492.73039341283
4,NW.HCA.FEMA.PC,ALB,4893.00627518049,4059.32053470641,4536.97205311183


In [ ]:
#Let's clean up this dataframe!!!
print(dfAC_clean.shape)
dfAC_clean = dfAC_clean.dropna()
for col in dfAC_clean.columns:
    dfAC_clean = dfAC_clean[dfAC_clean[col].str.startswith('..') == False]

print(dfAC_clean.shape)
dfAC_clean.head(50)

In [111]:
dfSe = pd.read_csv("data/seriesmetadata.csv")

In [ ]:
dfSe.head()

In [115]:
dfSe_clean = dfSe.drop(columns = ["Previous Indicator Code", "Previous Indicator Name"])
dfSe_clean.rename(columns = {"Code":"Series Code"}, inplace = True) #Named 'Series Code' in other dataset

In [116]:
dfSe_clean.head()

,Series Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...


In [ ]:
dfmdg = pd.read_csv("data/mdgdata.csv")
dfmdg.head()

In [ ]:
dfmdg.columns

In [108]:
selected_frame = dfmdg[['Country Name', 'Country Code', '1999 [YR1999]', '1993 [YR1993]']]

In [ ]:
print(selected_frame.shape)
dfmdg_clean = mask_frame.dropna()   #Remove NaN observations
print(dfmdg_clean.shape)
dfmdg_clean.head()

In [110]:
#Let's clean up this dataframe!!!
print(dfmdg_clean.shape)
for col in dfmdg_clean.columns:
    dfmdg_clean = dfmdg_clean[dfmdg_clean[col].str.startswith('..') == False]

print(dfmdg_clean.shape)
dfmdg_clean.head()

(34716, 4)
(12771, 4)


,Country Name,Country Code,1999 [YR1999],1993 [YR1993]
3,Afghanistan,AFG,157.71,164.1888
23,Afghanistan,AFG,821.408,1345.789
24,Afghanistan,AFG,0.0423325971841624,0.0891540360218903
32,Afghanistan,AFG,69.4990005493164,60.9059982299805
33,Afghanistan,AFG,29.4969997406006,24.1599998474121


In [206]:
cur, conn = create_db()

Database connection successful!
New database connection successful!


In [ ]:
cur.execute("DROP TABLE accountscountry")
conn.commit()
print("done")

In [180]:
#Let's Create first table (Account)

ac_table_create = (""" CREATE TABLE IF NOT EXISTS accountscountry(
id serial PRIMARY KEY,
country_code VARCHAR,
series_code VARCHAR,
year_95 numeric,
year_97 numeric,
year_99 numeric
)""")

In [181]:
#If this throws an error such as: "postgresql current transaction is aborted, commands ignored until end of transaction block", 
#then run these to close previous connections
# cur.close()
# conn.close()

cur.execute(ac_table_create)
conn.commit()
print("Table created successfully")

Table created successfully


In [178]:
#Now Series Table

acseries_table_create = ("""CREATE TABLE IF NOT EXISTS seriesdata(
series_code VARCHAR,
indicator_name VARCHAR,
long_definition VARCHAR,
source VARCHAR,
topic VARCHAR,
unit_of_measure VARCHAR,
periodicity VARCHAR,
reference_peoriod VARCHAR,
stat_concept VARCHAR
)""")

cur.execute(acseries_table_create)
conn.commit()
print("Table created successfully")

Table created successfully


In [179]:
#Now MDG Table
acmdg_table_create = ("""CREATE TABLE IF NOT EXISTS mdgdata(
country_name VARCHAR,
country_code VARCHAR,
year_99 numeric,
year_93 numeric
)""")
cur.execute(acmdg_table_create)
conn.commit()
print("Table created successfully")

Table created successfully


In [224]:
cur.close()
conn.close()
cur, conn = create_db()

Database connection successful!
New database connection successful!


### Time to insert data into the tables

In [211]:
ac_table_insert = (""" INSERT INTO accountscountry(
country_code,
series_code,
year_95,
year_97,
year_99)
VALUES (%s, %s, %s, %s, %s)
""")

In [229]:
#Now loop through the dataframe and insert all data

for i, row in dfAC_clean.iterrows():
    cur.execute(ac_table_insert, list(row))
conn.commit()
print("Items entered successfully.")

Items entered successfully.


In [223]:
acseries_table_insert = ("""INSERT INTO seriesdata(
series_code,
indicator_name,
long_definition,
source,
topic,
unit_of_measure,
periodicity,
reference_peoriod,
stat_concept)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [225]:
for i, row in dfSe_clean.iterrows():
    cur.execute(acseries_table_insert, list(row))
conn.commit()
print("Items entered successfully.")

Items entered successfully.


In [226]:
acmdg_table_insert = ("""INSERT INTO mdgdata(
country_name,
country_code,
year_99,
year_93)
VALUES (%s, %s, %s, %s)
""")

In [227]:
for i, row in dfmdg_clean.iterrows():
    cur.execute(acmdg_table_insert, list(row))
conn.commit()
print("Items entered successfully.")

Items entered successfully.


In [228]:
conn.close()